In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import sum, avg, rank, dense_rank
from pyspark.sql.window import Window
from pyspark.sql.functions import when, col
from pyspark.sql.functions import sum, avg, round as rnd, col
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank


In [0]:
# Accessing sales table from bronze schema under retail_poc catalog
silver_df = spark.table('retail_poc.silver.combined_sales_summary')


In [0]:
# silver_df.describe()

In [0]:
# display(silver_df)

In [0]:
#created a table for visualizing total sales per month for each store on the basis of aggregated Margin %
aggregated_df = silver_df.groupBy("Store_code", "store_name", "Region", "Month", "Year", "Season"
).agg(
    sum("Sale_QTY").alias("Units_Sold"),
    rnd(sum("COGS"), 2).alias("COGS"),
    rnd(sum("Net_Sales"), 2).alias("Total_Sales"),
    rnd(sum("Net_Margin"), 2).alias("Net_Margin"),
    rnd((sum("Net_Margin") / sum("Net_Sales")) * 100, 2).alias("agg_Margin_%"),
)



In [0]:
# %python
# # Select all necessary columns for the groupBy operation
# aggregated_df = silver_df.select(
#     "Store_code", "store_name", "Region", "Month", "Year", "Season", "Sale_QTY", "COGS", "Net_Sales", "Net_Margin"
# ).groupBy(
#     "Store_code", "store_name", "Region", "Month", "Year"
# ).agg(
#     sum("Sale_QTY").alias("Units_Sold"),
#     rnd(sum("COGS"), 2).alias("COGS"),
#     rnd(sum("Net_Sales"), 2).alias("Total_Sales"),
#     rnd(sum("Net_Margin"), 2).alias("Net_Margin"),
#     rnd((sum("Net_Margin") / sum("Net_Sales")) * 100, 2).alias("agg_Margin_%"),
# )

# # display(aggregated_df)

In [0]:
aggregated_df.write.mode("overwrite").saveAsTable("retail_poc.gold.store_sales_per_month")

In [0]:

daily_aggregated_df = silver_df.groupBy("Store_code","store_name", "Region", "Date", "Season").agg(
    sum("Sale_QTY").alias("Units_Sold"),
    rnd(sum("COGS"), 2).alias("COGS"),
    rnd(sum("Net_Sales"), 2).alias("Total_Sales"),
    rnd(sum("Net_Margin"), 2).alias("Net_Margin"),
    rnd((sum("Net_Margin") / sum("Net_Sales")) * 100, 2).alias("agg_Margin_%")
)


In [0]:
# daily_aggregated_df.display()

In [0]:
daily_aggregated_df.write.mode("overwrite").saveAsTable("retail_poc.gold.store_Sale_per_Day")

In [0]:

yearly_aggregated_df = silver_df.groupBy("Store_code","store_name", "Region", "Year").agg(
    sum("Sale_QTY").alias("Units_Sold"),
    rnd(sum("COGS"), 2).alias("COGS"),
    rnd(sum("Net_Sales"), 2).alias("Total_Sales"),
    rnd(sum("Net_Margin"), 2).alias("Net_Margin"),
    rnd((sum("Net_Margin") / sum("Net_Sales")) * 100, 2).alias("agg_Margin_%")
)



In [0]:
yearly_aggregated_df.write.mode("overwrite").saveAsTable("retail_poc.gold.store_Sale_per_Year")

In [0]:
# yearly_aggregated_df.display()

In [0]:
#To rank stores within each month-year group based on Total Sales, in descending order.
window_spec = Window.partitionBy("Month", "Year").orderBy(F.col("Total_Sales").desc())
ranked_df = aggregated_df.withColumn("Rank", dense_rank().over(window_spec))

In [0]:
ranked_df.write.format("delta").mode("overwrite").saveAsTable("retail_poc.gold.store_rank_permonth_peryear")

In [0]:
# ranked_df.display()

In [0]:
# distinct_ranks_df = ranked_df.select("Rank").distinct()
# display(distinct_ranks_df)

In [0]:
# aggregated_df.display()

In [0]:

#Aggregate total sales per store per year
df = aggregated_df.groupBy("store_name", "Year", "Region", "Season").agg(
    F.sum("Total_Sales").alias("Yearly_Total_Sales")
)

#Define window to rank stores within each year
window_spec = Window.partitionBy("Year").orderBy(F.col("Yearly_Total_Sales").desc())

#Add rank column
updated_ranked_df = df.withColumn("Rank", dense_rank().over(window_spec))


In [0]:
# updated_ranked_df.display()

In [0]:
updated_ranked_df.write.mode("overwrite").saveAsTable("retail_poc.gold.yearly_store_rank")

In [0]:
#Aggregate total sales per store for each month and year
monthly_agg_df = aggregated_df.groupBy("store_name", "Month", "Year", "Season").agg(
    F.sum("Total_Sales").alias("Monthly_Total_Sales")
)

#Define window to rank stores within each month-year
window_spec = Window.partitionBy("Month", "Year").orderBy(F.col("Monthly_Total_Sales").desc())

#Add rank column
ranked_monthly_df = monthly_agg_df.withColumn("Rank", dense_rank().over(window_spec))


In [0]:
# ranked_monthly_df.display()


In [0]:
ranked_monthly_df.write.mode("overwrite").saveAsTable("retail_poc.gold.ranked_monthly")

In [0]:
product_profit_df = silver_df.groupBy("EAN", "Category", "Sub_Category", "Brand_code", "brand_as_per_master", "Gender", "Month", "Year", "Season").agg(
    sum("Sale_QTY").alias("Units_Sold"),
    sum("Net_Margin").alias("Total_Margin"),
    sum("Sale_iv").alias("Total_Sales"),
    sum("COGS").alias("Total_COGS"),
    (sum("Net_Margin") / sum("Net_Sales") * 100).alias("agg_Margin_%")
)

In [0]:
# product_profit_df.display()



In [0]:

product_profit_margin_df = product_profit_df.withColumn(

    "Margin_Segment",
    when(col("agg_Margin_%") >= 60, "High")
    .when(col("agg_Margin_%") >= 30, "Medium")
    .otherwise("Low")
)


In [0]:
# product_profit_margin_df.display()

In [0]:
product_profit_margin_df.write.format("delta").mode("overwrite").saveAsTable("retail_poc.gold.product_profit_margin")

In [0]:


# avg_sales_margin_month = silver_df.groupBy("Store_code","store_name", "Year", "Month").agg(
#     avg("Net_Sales").alias("avg_monthly_sales"),
#     avg("Net_Margin").alias("avg_monthly_margin")
# )
# avg_sales_margin_month.show()


In [0]:
# region_sales_margin = silver_df.groupBy("Region").agg(
#     sum("Net_Sales").alias("total_sales"),
#     sum("Net_Margin").alias("total_margin")
# )
# region_sales_margin.show()
